# Sistem Penjadwalan Praktikum Otomatis menggunakan Algoritma Genetika dan Tabu Search.

#### Import Library

In [5]:
from collections import namedtuple

## Inisiasi Data

In [6]:
# Namedtuple for Semester model
SemesterTuple = namedtuple('SemesterTuple', ['id', 'name'])
# Namedtuple for Laboratory model
LaboratoryTuple = namedtuple('LaboratoryTuple', ['id', 'name'])
# Namedtuple for Module model
ModuleTuple = namedtuple('ModuleTuple', ['id', 'name', 'start_date', 'end_date', 'laboratory_id', 'semester_id'])
# Namedtuple for Chapter model
ChapterTuple = namedtuple('ChapterTuple', ['id', 'name', 'module_id'])
# Namedtuple for Group model
GroupTuple = namedtuple('GroupTuple', ['id', 'name', 'module_id'])
# Namedtuple for Participant model
ParticipantTuple = namedtuple('ParticipantTuple', ['id', 'name', 'nim', 'semester_id', 'regular_schedule'])
# Namedtuple for Assistant model
AssistantTuple = namedtuple('AssistantTuple', ['id', 'name', 'nim', 'laboratory_id', 'semester_id', 'regular_schedule', 'prefered_schedule'])
# Namedtuple for GroupMembership model
GroupMembershipTuple = namedtuple('GroupMembershipTuple', ['id', 'participant_id', 'group_id'])
# Namedtuple for AssistantMembership model
AssistantMembershipTuple = namedtuple('AssistantMembershipTuple', ['id', 'assistant_id', 'module_id'])

In [7]:
# Fillin the data
semesters = [SemesterTuple(1, 'Ganjil 22/23'), SemesterTuple(2, 'Genap 22/23')]
laboratories = [
    LaboratoryTuple(id=1, name='Dasar Elektro'),
    LaboratoryTuple(id=2, name='Kendali'),
    LaboratoryTuple(id=3, name='Tenaga'),
    LaboratoryTuple(id=4, name='Telekomunikasi'),
    LaboratoryTuple(id=5, name='Komputer'),
]
modules = [ModuleTuple(id=1, name='Pengukuran Listrik', start_date='2022-09-12', end_date='2022-11-26', laboratory_id=1, semester_id=1)]
chapters = [
    ChapterTuple(id=1, name='U-1', module_id=1),
    ChapterTuple(id=2, name='U-2', module_id=1),
    ChapterTuple(id=3, name='U-3', module_id=1),
    ChapterTuple(id=4, name='U-4', module_id=1),
    ChapterTuple(id=5, name='U-5', module_id=1),
    ChapterTuple(id=6, name='U-6', module_id=1),
    ChapterTuple(id=7, name='U-7', module_id=1),
    ChapterTuple(id=8, name='U-8', module_id=1),
]
groups = [
    GroupTuple(id=1, name='PL-1', module_id=1),
    GroupTuple(id=2, name='PL-2', module_id=1),
    GroupTuple(id=3, name='PL-3', module_id=1),
    GroupTuple(id=4, name='PL-4', module_id=1),
    GroupTuple(id=5, name='PL-5', module_id=1),
    GroupTuple(id=6, name='PL-6', module_id=1),
    GroupTuple(id=7, name='PL-7', module_id=1),
    GroupTuple(id=8, name='PL-8', module_id=1),
    GroupTuple(id=9, name='PL-9', module_id=1),
    GroupTuple(id=10, name='PL-10', module_id=1),
    GroupTuple(id=11, name='PL-11', module_id=1),
    GroupTuple(id=12, name='PL-12', module_id=1),
    GroupTuple(id=13, name='PL-13', module_id=1),
    GroupTuple(id=14, name='PL-14', module_id=1),
    GroupTuple(id=15, name='PL-15', module_id=1),
    GroupTuple(id=16, name='PL-16', module_id=1),
    GroupTuple(id=17, name='PL-17', module_id=1),
    GroupTuple(id=18, name='PL-18', module_id=1),
    GroupTuple(id=19, name='PL-19', module_id=1),
    GroupTuple(id=20, name='PL-20', module_id=1),
    GroupTuple(id=21, name='PL-21', module_id=1),
    GroupTuple(id=22, name='PL-22', module_id=1),
    GroupTuple(id=23, name='PL-23', module_id=1),
    GroupTuple(id=24, name='PL-24', module_id=1),
    GroupTuple(id=25, name='PL-25', module_id=1),
    GroupTuple(id=26, name='PL-26', module_id=1),
    GroupTuple(id=27, name='PL-27', module_id=1),
    GroupTuple(id=28, name='PL-28', module_id=1),
    GroupTuple(id=29, name='PL-29', module_id=1),
    GroupTuple(id=30, name='PL-30', module_id=1),
    GroupTuple(id=31, name='PL-31', module_id=1),
    GroupTuple(id=32, name='PL-32', module_id=1),
    GroupTuple(id=33, name='PL-33', module_id=1),
    GroupTuple(id=34, name='PL-34', module_id=1),
    GroupTuple(id=35, name='PL-35', module_id=1),
    GroupTuple(id=36, name='PL-36', module_id=1),
]
